<a href="https://colab.research.google.com/github/rahiakela/transformers-research-and-practice/blob/main/developing-kaggle-notebooks/10-GenAI/04_simple_sequential_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'llama-2/pytorch/7b-chat-hf/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F3093%2F4298%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240130%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240130T125344Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Db55d6c3e3f06c3f84751ee02016215a0da55a152645454ad756e7e55fe6e839bad7ee7e510fbf04c14a47397be525189f05fa5bf86e47591442370a3dd9d6da916ad620c50db3e963aa5427a963ea523d1ff0c6e34737fb3f3788c491d26be98c1f19852add1d8c03b9d3576c19230103eceed783955745a094e146aaa43ad13d3b87735ecab6ef4a92812b40e73fff20a6194321df14a48881e1d02c42543ad25239595567bfb60b00a8863d48e356bfe1ec5f2c009369c612a7687c6dadbc586a00dc99a66f23ccf99fe5a428ca3c85bd4c9bf91471986149474b0c06d2b2728ab1e5f7138b0d7d6402e7e2f23ee0f39bf0d8d96250465af59096cc49e9aa6'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Introduction


## Objective  

Use Llama 2 and Langchain to create a multi-step task chain.

## Models details  

* **Model #1**: Llama 2  
* **Variation**: 7b-chat-hf    
* **Version**: V1  
* **Framework**: PyTorch  


LlaMA 2 model is pretrained and fine-tuned with 2 Trillion tokens and 7 to 70 Billion parameters which makes it one of the powerful open source models. It is a highly improvement over LlaMA 1 model.

# InstalIing, imports, utils

Install packages.

In [ ]:
!pip install transformers==4.33.0 accelerate==0.22.0 einops==0.6.1 langchain==0.0.300 xformers==0.0.21 \
bitsandbytes==0.41.1 sentence_transformers==2.2.2 chromadb==0.4.12

Import packages.

In [ ]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time

from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain import PromptTemplate


## Initialize model, tokenizer, query pipeline  

Define the model, the device, and the bitsandbytes configuration.

In [ ]:
model_1_id = '/kaggle/input/llama-2/pytorch/7b-chat-hf/1'

model_2_id = '/kaggle/input/llama-2/pytorch/13b-chat-hf/1'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

Prepare the model and the tokenizer.  

We perform this operation for both models (7b & 13b).

In [ ]:
time_1 = time()
model_1_config = transformers.AutoConfig.from_pretrained(
    model_1_id,
)
model_1 = transformers.AutoModelForCausalLM.from_pretrained(
    model_1_id,
    trust_remote_code=True,
    config=model_1_config,
    quantization_config=None,
    device_map='auto',
)
tokenizer_1 = AutoTokenizer.from_pretrained(model_1_id)
time_2 = time()
print(f"Prepare model #1, tokenizer: {round(time_2-time_1, 3)} sec.")

Define a pipeline.

In [ ]:
time_1 = time()
query_pipeline_1 = transformers.pipeline(
        "text-generation",
        model=model_1,
        tokenizer=tokenizer_1,
        torch_dtype=torch.float16,
        device_map="auto",)
time_2 = time()
print(f"Prepare pipeline #1: {round(time_2-time_1, 3)} sec.")

llm_1 = HuggingFacePipeline(pipeline=query_pipeline_1)



We test it by running a simple query.

In [ ]:
# checking model #1
llm_1(prompt="What is the most popular food in France for tourists? Just return the name of the food.")

# Define and execute the sequential chain


We define a chain with two tasks in sequence.  
The input for the second step is the output of the first step.  

In [ ]:
def sequential_chain(country, llm):
    """
    Args:
        country: country selected
    Returns:
        None
    """
    time_1 = time()
    template = "What is the most popular food in {country} for tourists? Just return the name of the food."

    #  first task in chain
    first_prompt = PromptTemplate(

    input_variables=["country"],

    template=template)

    chain_one = LLMChain(llm = llm, prompt = first_prompt)

    # second step in chain
    second_prompt = PromptTemplate(

    input_variables=["food"],

    template="What are the top three ingredients in {food}. Just return the answer as three bullet points.",)

    chain_two = LLMChain(llm=llm, prompt=second_prompt)

    # combine the two steps and run the chain sequence
    overall_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)
    overall_chain.run(country)
    time_2 = time()
    print(f"Run sequential chain: {round(time_2-time_1, 3)} sec.")

Test the sequence with Llama v2 **7b** chat HF model.

In [ ]:
final_answer = sequential_chain("France", llm_1)

Test the sequence with Llama v2 **7b** chat HF model.

In [ ]:
final_answer = sequential_chain("Italy", llm_1)